In [1]:
library(tidyverse)
library(tsibble)

Warning message:
"package 'ggplot2' was built under R version 4.3.2"
── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: 'tsibble'


The following object is masked from 'package:lubridate':

    interval


The following objects are masked from 'package:base':

    intersect, setdiff, union




### Import the housing data

In [2]:
df <- read.csv('monthly_housing_inventory.csv')
df %>% dim()

[1] 288385     10

In [3]:
df %>% distinct() %>% dim()

[1] 288385     10

In [4]:
df %>% head()

,month_date_yyyymm,county_name,median_listing_price,active_listing_count,new_listing_count,pending_listing_count,median_listing_price_per_square_foot,median_square_feet,average_listing_price,total_listing_count
,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,202403,"hartley, tx",425000,12,0,1,289,3024,444336,12
2,202403,"rock, mn",309900,21,8,13,136,2016,397205,35
3,202403,"carter, ok",199000,165,56,59,127,1633,306719,222
4,202403,"mckean, pa",169000,48,12,25,89,1898,187270,71
5,202403,"etowah, al",248950,236,128,67,124,1894,296370,303
6,202403,"dawson, ga",679450,135,76,80,227,2802,814852,211


### Extract county, state, and create date variable
The `county_name` contains the `county` and `state`. Use `regex` to extract those into separate fields. Convert the `month_date_yyyymm` from an integer to a `monthly date`.


In [5]:
df <-  df %>% 
mutate(county = str_extract(county_name, '^(.*)?, ([a-z]{2})$', group=1)) %>%
mutate(state = str_extract(county_name, '^(.*)?, ([a-z]{2})$', group=2)) %>%
mutate(month_date = tsibble::yearmonth(as.character(month_date_yyyymm), '%Y%m'))

df %>% head()

,month_date_yyyymm,county_name,median_listing_price,active_listing_count,new_listing_count,pending_listing_count,median_listing_price_per_square_foot,median_square_feet,average_listing_price,total_listing_count,county,state,month_date
,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<mth>
1,202403,"hartley, tx",425000,12,0,1,289,3024,444336,12,hartley,tx,2024 Mar
2,202403,"rock, mn",309900,21,8,13,136,2016,397205,35,rock,mn,2024 Mar
3,202403,"carter, ok",199000,165,56,59,127,1633,306719,222,carter,ok,2024 Mar
4,202403,"mckean, pa",169000,48,12,25,89,1898,187270,71,mckean,pa,2024 Mar
5,202403,"etowah, al",248950,236,128,67,124,1894,296370,303,etowah,al,2024 Mar
6,202403,"dawson, ga",679450,135,76,80,227,2802,814852,211,dawson,ga,2024 Mar


In [8]:
df_ts <- df %>% as_tsibble(index=month_date, key=c(state, county))

In [9]:
df_ts %>% head()

month_date_yyyymm,county_name,median_listing_price,active_listing_count,new_listing_count,pending_listing_count,median_listing_price_per_square_foot,median_square_feet,average_listing_price,total_listing_count,county,state,month_date
<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<mth>
201607,"aleutians east, ak",485000,3,0,NA,93,4100,394833,3,aleutians east,ak,2016 Jul
201608,"aleutians east, ak",342250,4,0,NA,87,3285,308350,4,aleutians east,ak,2016 Aug
201609,"aleutians east, ak",342250,4,0,NA,87,3285,307350,4,aleutians east,ak,2016 Sep
201610,"aleutians east, ak",199500,4,0,NA,93,2469,281880,4,aleutians east,ak,2016 Oct
201611,"aleutians east, ak",342250,4,0,NA,106,3285,336125,4,aleutians east,ak,2016 Nov
201612,"aleutians east, ak",342250,4,0,NA,106,3285,336125,4,aleutians east,ak,2016 Dec
